### Working with `boto3` - Demo

In [1]:
import boto3
import pandas as pd
import datetime

We need to create a session to use our default profile we created earlier. (Change the name, if you have named it otherwise)

In [2]:
sess = boto3.Session(profile_name='default')

To make sure we are in the profile that we want, we'll print the details.

In [3]:
print("The AWS profile active is {}".format(sess.profile_name))
print("The session active region is {}".format(sess.region_name))

The AWS profile active is default
The session active region is us-east-1


Since we need to interact with EC2 instances, we create the ec2 object as below: 

In [4]:
ec2 = sess.resource('ec2')

### 1. Create `EC2` instance using `boto3`

- First thing before creating any instance is to create `KeyPair` assigned to that instance.

In [6]:
response = ec2.create_key_pair(KeyName='demo2')

In [8]:
print("The response recieved is {}".format(response))

- Saving the key to `.pem file` to local system.

In [9]:
outfile = open('/home/pratos/.ssh/file1.pem','w')
KeyPairOut = str(response.key_material)
outfile.write(KeyPairOut)

1670

In [9]:
!ls ~/.ssh/

file1.pem		   google_compute_known_hosts  id_rsa_azure.pub
file.pem		   greyatom.pem		       id_rsa.pub
google_compute_engine	   id_rsa		       known_hosts
google_compute_engine.pub  id_rsa_azure		       webapp-a87226add156.json


- Creating an instance using a Deep Learning AMI

Using Amazon's Deep Learning AMI [us-east-1 : ami-e47723f2] ([Link](https://aws.amazon.com/marketplace/pp/B01M0AXXQB))

In [10]:
dl_instance1 = ec2.create_instances(ImageId='ami-e47723f2', 
                                    MinCount=1, #Minimum number of instances 
                                    MaxCount=1, #Maximum number of instances
                                    KeyName="demo1", #Key that we generated earlier
                                    InstanceType="t2.micro") 

__NOTE:__ Use the AMI Image that is specific for the region. Also, keys generated are for a specific region. Keys from one region are not transferrable to other regions, you'll get errors (Key not available).

For the next set of instructions, we need to get the instance id, follow the steps to get one:

In [11]:
print("The instance id: {}".format(dl_instance1[0].id))

The instance id: i-0e1ae1bb317f7b8f2


### 2. Stop, Start & Reboot the EC2 instance

There are two modes in which we can execute: `start_instances` & `stop_instances`:

- `DryRun=True`: Dry run to verify whether the AWS account executing the action has permissions to do it.
- `DryRun=False`: AWS account executing the action has full permissions to do it & actual execution happens.

Let's try it out, since we have created this EC2 instance we will have full permissions:

In [12]:
ec2_client = boto3.client('ec2')

In [13]:
instance_id = dl_instance1[0].id

__We will try to stop and start the instance, with `DryRun=False`:__

In [16]:
try:
    response = ec2_client.stop_instances(InstanceIds=[instance_id], DryRun=False)
    print(response)
except ClientError as e:
    print(e)

{'StoppingInstances': [{'PreviousState': {'Name': 'running', 'Code': 16}, 'InstanceId': 'i-08b0593cb0cb54f5e', 'CurrentState': {'Name': 'stopping', 'Code': 64}}], 'ResponseMetadata': {'RequestId': '4556624d-f9c0-4da4-8313-fa20ed2ab33d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'server': 'AmazonEC2', 'date': 'Sat, 09 Sep 2017 20:11:02 GMT', 'content-length': '540'}, 'RetryAttempts': 0}}


__Starting the instance again:__

(NOTE: Wait till the instance is stopped/started before starting/stopping)

In [29]:
try:
    response = ec2_client.start_instances(InstanceIds=[instance_id], DryRun=False)
    print(response)
except ClientError as e:
    print(e)

{'ResponseMetadata': {'RequestId': 'ae6918a9-5f33-41a1-ba46-984d93babb8c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'server': 'AmazonEC2', 'date': 'Sat, 09 Sep 2017 20:24:17 GMT', 'content-length': '540'}, 'RetryAttempts': 0}, 'StartingInstances': [{'PreviousState': {'Name': 'stopped', 'Code': 80}, 'InstanceId': 'i-08b0593cb0cb54f5e', 'CurrentState': {'Name': 'pending', 'Code': 0}}]}


__Rebooting the instance:__

We do have the same attribute, `DryRun` for __rebooting__ the instances:

(We'll directly reboot the instances)

In [33]:
try:
    response = ec2_client.reboot_instances(InstanceIds=[instance_id], DryRun=False)
    print("Success: {}".format(response))
except ClientError as e:
    print("Error: {}".format(e))

Success {'ResponseMetadata': {'HTTPHeaders': {'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'date': 'Sat, 09 Sep 2017 14:25:49 GMT', 'server': 'AmazonEC2', 'content-type': 'text/xml;charset=UTF-8'}, 'RetryAttempts': 0, 'RequestId': '9fdaa44d-e6d9-43ae-a7ce-b9e04d8813e8', 'HTTPStatusCode': 200}}


### 3. Terminate an EC2 Instance

Terminating EC2 instance using `client` interface.

In [14]:
try:
    response = ec2_client.terminate_instances(InstanceIds=[instance_id], DryRun=False)
    print("Success: {}".format(response))
except ClientError as e:
    print("Error: {}".format(e))

Success: {u'TerminatingInstances': [{u'InstanceId': 'i-0e1ae1bb317f7b8f2', u'CurrentState': {u'Code': 32, u'Name': 'shutting-down'}, u'PreviousState': {u'Code': 0, u'Name': 'pending'}}], 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '80c981fc-2e87-487e-9aa6-6ecc749b4ade', 'HTTPHeaders': {'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'server': 'AmazonEC2', 'content-type': 'text/xml;charset=UTF-8', 'date': 'Sun, 10 Sep 2017 08:20:47 GMT'}}}


### 4. Creating S3 Buckets

In [16]:
import boto3

s3 = boto3.client('s3')

In [38]:
s3.create_bucket(Bucket='greyatom-demo')

{'Location': '/greyatom-demo',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Sat, 09 Sep 2017 19:44:57 GMT',
   'location': '/greyatom-demo',
   'server': 'AmazonS3',
   'x-amz-id-2': 'Aodegn6SUaqX/McWWTT5WeOAS22RQcn2Z6N1kktmKcSXzB01CqAeoY2xqsP0C3wo+j44MiFpgYo=',
   'x-amz-request-id': '0E895C25E6615118'},
  'HTTPStatusCode': 200,
  'HostId': 'Aodegn6SUaqX/McWWTT5WeOAS22RQcn2Z6N1kktmKcSXzB01CqAeoY2xqsP0C3wo+j44MiFpgYo=',
  'RequestId': '0E895C25E6615118',
  'RetryAttempts': 0}}

Checking in the S3 dashboard we have successfully created the `greyatom-demo` bucket.

![]()

### 5. Uploading file to a bucket specified

In [17]:
!ls ./data/

test_table.csv	user_table.csv


- We'll list the buckets we have:

In [18]:
response = s3.list_buckets()
bucketlist = [resp.get('Name') for resp in response.get('Buckets')]
print("The list of buckets:{}".format(bucketlist))

The list of buckets:['greyatom-demo', 'lambda-test-dev-serverlessdeploymentbucket-1cwwgbpniigsx']


We'll put both the .csv files in the `greyatom-demo` bucket.

In [31]:
import glob
import re
filenames = glob.glob('./data/*.csv')

In [26]:
filenames

['./data/test_table.csv', './data/user_table.csv']

In [27]:
bucket_name = 'greyatom-demo'

In [35]:
for file in filenames:
    filename = re.split(r'/', file)
    print("Uploading file: {}".format(filename[len(filename)-1]))
    s3.upload_file(file, bucket_name, filename[len(filename)-1])

Uploading file: test_table.csv
Response: None
Uploading file: user_table.csv
Response: None


### 6. List all the files in a bucket

In [19]:
bucket_name = 'greyatom-demo'

In [20]:
s3_resource = boto3.resource('s3')

my_bucket = s3_resource.Bucket(bucket_name)

for obj in my_bucket.objects.all():
    print(obj.key)

./data/test_table.csv
./data/user_table.csv
daily-minimum-temperatures.csv
test_table.csv
user_table.csv


### 7. Downloading a file from the bucket

The example below tries to download an S3 object to a file. If the service returns a 404 error, it prints an error message indicating that the object doesn't exist.

In [53]:
bucket_name = 'greyatom-demo'
key = 'daily-minimum-temperatures.csv'

s3 = boto3.resource('s3')

try:
    s3.Bucket(bucket_name).download_file(key, './min_daily_temp.csv')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [55]:
!ls ./

data  day28_aws_boto3_demo.ipynb  images  min_daily_temp.csv


### 8. Deleting an S3 bucket

We need to delete all the `keys` (everything saved in the bucket) before the `S3` bucket itself can be deleted.

In [21]:
bucket_name = 'greyatom-demo'

In [22]:
bucket = s3_resource.Bucket(bucket_name)

In [23]:
for key in bucket.objects.all():
    key.delete()
    print("{} deleted...".format(key))

bucket.delete()

s3.ObjectSummary(bucket_name='greyatom-demo', key=u'./data/test_table.csv') deleted...
s3.ObjectSummary(bucket_name='greyatom-demo', key=u'./data/user_table.csv') deleted...
s3.ObjectSummary(bucket_name='greyatom-demo', key=u'daily-minimum-temperatures.csv') deleted...
s3.ObjectSummary(bucket_name='greyatom-demo', key=u'test_table.csv') deleted...
s3.ObjectSummary(bucket_name='greyatom-demo', key=u'user_table.csv') deleted...


{'ResponseMetadata': {'HTTPHeaders': {'date': 'Sun, 10 Sep 2017 08:30:53 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'u33q7yMsT2ddoQvee/aauxO/Fq9OZwN1zqOzPNwIdnU91W0opybLnZvkAUQFQerNd/V2LOgxTNo=',
   'x-amz-request-id': 'E18C44C76BB99876'},
  'HTTPStatusCode': 204,
  'HostId': 'u33q7yMsT2ddoQvee/aauxO/Fq9OZwN1zqOzPNwIdnU91W0opybLnZvkAUQFQerNd/V2LOgxTNo=',
  'RequestId': 'E18C44C76BB99876',
  'RetryAttempts': 0}}

Listing the buckets to confirm, whether it was really deleted

In [24]:
response = s3.list_buckets()
bucketlist = [resp.get('Name') for resp in response.get('Buckets')]
print("The list of buckets:{}".format(bucketlist))

The list of buckets:['lambda-test-dev-serverlessdeploymentbucket-1cwwgbpniigsx']


Success!